In [2]:
# 1) Variance reduction using Control Variates

import numpy as np
from scipy import stats
 
## BSM closed-form solution (for call) ##
def bsm_call_value(S0, K, T, r, sigma):
    d1= (np.log(S0/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2= (np.log(S0/K)+(r-0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    call_value = (S0*stats.norm.cdf(d1,0.0,1.0) - K*np.exp(-r*T)*stats.norm.cdf(d2,0.0,1.0))
    return call_value

    
np.random.seed(0)
N=int(1e6)
 
S0=100 ; K=105 ; T=2 ; r=0.05 ; sig=0.2

M = 8  
dt = T/M

mu = np.zeros(M)
A = np.zeros((M,M))
for i in range(M):
    for j in range(M):
        if i>=j:
            A[i,j] = np.sqrt(dt)

mu = np.zeros(M)
eye_M = np.eye(M)    
Z = np.random.multivariate_normal(mu,eye_M,N)      
W = np.dot(A,Z.T)
W = W.T
S_t = np.zeros((N,M))
t = np.zeros(M)

for j in range(M):
    t[j] = dt*(j+1)
    S_t[:,j] = S0*np.exp((r-sig*sig/2)*t[j]+sig*W[:,j])

S_t_bar=np.zeros(N)  # 산술 평균
S_t_gbar=np.zeros(N) # geometric 평균

for k in range(N):
    S_t_bar[k]=np.mean(S_t[k,:]) # 산술평균 계산
    S_t_gbar[k]=(S0*np.prod(S_t[k,:]))**(1/(M+1)) # 기하평균 계산

h_T  = (S_t_bar-K)*(S_t_bar>K)   # 일반적 asian call option pay-off
CV_T = (S_t_gbar-K)*(S_t_gbar>K) # geometric asian call option pay-off
h_0  = np.exp(-r*T)*h_T
CV_0 = np.exp(-r*T)*CV_T  # 부산물

# closed-form solution of a version of Asian call (= geometric asian call)
# control Variate(IV) 참조
sig_z = sig*np.sqrt((2*M+1)/(6*M+6))
rho   = 0.5*((r-sig*sig/2)+sig_z*sig_z)
E_CV_0 = np.exp((rho-r)*T)*bsm_call_value(S0, K, T, rho, sig_z) #


# control variate method
cov_mat = np.cov(h_0,CV_0)
c_star = - cov_mat[0,1]/cov_mat[1,1] # C*

h_0_c = h_0 + c_star*(CV_0-E_CV_0)

h_0_e = np.mean(h_0_c) ; h_0_std = np.std(h_0_c)

print("h_0 =",h_0_e) 
print("95% confidence interval =",[h_0_e-1.96*h_0_std/np.sqrt(N),h_0_e+1.96*h_0_std/np.sqrt(N)])

h_0 = 7.2293873908272
95% confidence interval = [7.228221305965206, 7.230553475689194]


In [4]:
# 2) Variance reduction using Antithetic variation

###########################################
##  Black-Scholes-Merton Option Pricing  ##
##  written by Baeho Kim                 ##
###########################################

import numpy as np
from scipy import stats

## BSM closed-form solution (for call) ##
def bsm_call_value(S0, K, T, r, sigma):
    d1= (np.log(S0/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2= (np.log(S0/K)+(r-0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    call_value = (S0*stats.norm.cdf(d1,0.0,1.0) - K*np.exp(-r*T)*stats.norm.cdf(d2,0.0,1.0))
    return call_value


## BSMOPM parameter setting ##
S_0= 100.
K= 105.
T= 2.
r= 0.05
sigma= 0.2

## Monte Carlo Simulation with Antithetic Variates ##
N = int(1e8)         # Number of sample paths
W_1_T = np.random.normal(loc=0.0, scale=np.sqrt(T), size=N)
W_2_T = -W_1_T

S_1_T = S_0*np.exp((r-0.5*sigma*sigma)*T + sigma*W_1_T)
S_2_T = S_0*np.exp((r-0.5*sigma*sigma)*T + sigma*W_2_T)

C_1_0 = np.exp(-r*T)*(S_1_T-K)*(S_1_T > K)
C_2_0 = np.exp(-r*T)*(S_2_T-K)*(S_2_T > K)
C_0 = 0.5*(C_1_0 + C_2_0)

print('Estimated Call Price (95% C.I.) = ',np.mean(C_0),' +- ',1.96*np.std(C_0)/np.sqrt(N))
print('True Call Price = ', bsm_call_value(S_0,K,T,r,sigma))

Estimated Call Price (95% C.I.) =  13.639488506516521  +-  0.002240720325430203
True Call Price =  13.639615096767713


In [7]:
# 3) conditional Monte Carlo

import numpy as np

np.random.seed(0)

N = int(1e5)

Y = np.random.exponential(1.0, N)
Z = np.random.exponential(0.5, N)

plain_MC = (Y+Z>4)
conditional_MC = np.exp(-(4-Z))*(Z<=4)+np.ones(N)*(Z>4)

print(np.mean(plain_MC), 1.96*np.std(plain_MC)/np.sqrt(N))
print(np.mean(conditional_MC), 1.96*np.std(conditional_MC)/np.sqrt(N))


0.03703 0.0011704134559657109
0.036398840810640855 0.00025480199574543275


In [9]:
# 4) importance sampling
# --> likelihood ratio가 좋아야 적용가능 (이 부분이 pain-point)
# variance reduction 효과만으로 따지면 제일 베스트

import numpy as np
 
np.random.seed(0)
N  = int(1e7)
mu = 25

# crude MC
X_MC = np.random.randn(N)
theta_MC = (X_MC>mu)

theta_MC_e   = np.mean(theta_MC)
theta_MC_std = np.std(theta_MC)

print("theta_MC_e =",theta_MC_e) 
print("95% confidence interval =",[theta_MC_e-1.96*theta_MC_std/np.sqrt(N),theta_MC_e+1.96*theta_MC_std/np.sqrt(N)])


# importance sampling (crude MC대비 무한대배 정교함, but 수학적 모델링 필요)
X_IS = np.random.randn(N) + mu
theta_IS = np.exp(-mu*X_IS+mu*mu/2)*(X_IS>mu)

theta_IS_e   = np.mean(theta_IS)
theta_IS_std = np.std(theta_IS)
    
print("theta_IS_e =",theta_IS_e) 
print("95% confidence interval =",[theta_IS_e-1.96*theta_IS_std/np.sqrt(N),theta_IS_e+1.96*theta_IS_std/np.sqrt(N)])

theta_MC_e = 0.0
95% confidence interval = [0.0, 0.0]
theta_IS_e = 3.0500060098140967e-138
95% confidence interval = [3.0395783155760526e-138, 3.060433704052141e-138]
